In [1]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Activation, GaussianNoise
from keras.models import load_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time
import datetime
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.models import Sequential
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

Using TensorFlow backend.


# Final.csv Contain edited csv file

In [2]:
df = pd.read_csv('Final.csv',error_bad_lines=False)

In [ ]:
temp = df['usage'].isnull()
index = []
for i in range(len(df)):
    if temp[i]:
        index.append(df['files'][i])

In [61]:
index

[]

In [62]:
df.usage.value_counts()

Casual          31697
Sports           3952
Ethnic           3073
Formal           2292
Smart Casual       67
Party              26
Travel             20
Name: usage, dtype: int64

In [63]:
df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,files
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,53759.jpg


# Below I have created and developed CNN model to predict usage

In [3]:
X = df['files']
y= df['usage']

In [5]:
y = y.astype('str')

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                            zoom_range = 0.3
                            )

In [8]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [9]:
X = pd.concat([X_train, y_train], axis=1)

In [ ]:
test = pd.contact([X_test, y_test])

In [14]:
test = test.reset_index()
test.drop(['index'], axis =1, inplace = True)
test.head()
X = X.reset_index()
X.drop(['index'], axis = 1, inplace = True)
X.head()

,files,usage
0,21987.jpg,Casual
1,6755.jpg,Sports
2,35684.jpg,Casual
3,19098.jpg,Ethnic
4,19431.jpg,Casual


In [20]:
train_data = train_datagen.flow_from_dataframe(dataframe=X, directory='images/',x_col = 'files', y_col = 'usage', target_size = (64,64))

Found 27555 validated image filenames belonging to 7 classes.


In [21]:
test_data = test_datagen.flow_from_dataframe(dataframe=test, directory='images/',x_col = 'files', y_col = 'usage', target_size = (64,64))

Found 13572 validated image filenames belonging to 7 classes.


In [22]:
cnn = Sequential()

In [23]:
cnn.add(Conv2D(filters = 16, kernel_size = (3,3) ,strides = (1,1), input_shape = (64,64,3), activation = 'relu'))
cnn.add(Conv2D(filters = 32, kernel_size = (3,3) ,strides = (1,1), input_shape = (64,64,16), activation = 'relu'))
cnn.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
cnn.add(Flatten())
cnn.add(Dense(80, activation = 'relu'))
cnn.add(Dropout(0.4))
cnn.add(Dense(7,activation = 'softmax'))


In [24]:
cnn.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
start = time.time()
cnn.fit_generator(
    train_data,
    epochs=15,
    validation_data=test_data,
)
end = time.time()
print('Processing time:',(end - start)/60)

Epoch 1/15
862/862 [==============================] - 497s 576ms/step - loss: 0.1926 - accuracy: 0.9364 - val_loss: 0.4717 - val_accuracy: 0.9534
Epoch 2/15
862/862 [==============================] - 387s 449ms/step - loss: 0.1456 - accuracy: 0.9482 - val_loss: 0.2781 - val_accuracy: 0.9441
Epoch 3/15
862/862 [==============================] - 422s 490ms/step - loss: 0.1378 - accuracy: 0.9506 - val_loss: 0.1961 - val_accuracy: 0.9591
Epoch 4/15
862/862 [==============================] - 413s 479ms/step - loss: 0.1344 - accuracy: 0.9529 - val_loss: 0.1691 - val_accuracy: 0.9609
Epoch 5/15
862/862 [==============================] - 432s 501ms/step - loss: 0.1316 - accuracy: 0.9546 - val_loss: 0.0550 - val_accuracy: 0.9588
Epoch 6/15
862/862 [==============================] - 426s 494ms/step - loss: 0.1287 - accuracy: 0.9552 - val_loss: 0.3654 - val_accuracy: 0.9605
Epoch 7/15
862/862 [==============================] - 435s 505ms/step - loss: 0.1283 - accuracy: 0.9556 - val_loss: 0.2835 -

In [124]:
cnn.save('usage_model.h5')

In [22]:

model = load_model('usage_model.h5')

# Calculating prediction accurary

In [22]:
true_classes = test_data.classes
class_labels = list(test_data.class_indices.keys())   

In [23]:
img_class = model.predict(test_data, steps = test_data.samples / test_data.batch_size )

In [24]:
predicted_classes = np.argmax(img_class, axis=1)

In [25]:
confusion = confusion_matrix(true_classes, predicted_classes)
print(confusion)

[[8728  640  401    0    0  693    0]
 [ 838   53   50    0    0   58    0]
 [ 628   49   28    0    0   46    0]
 [   7    0    0    0    0    1    0]
 [  14    4    0    0    0    1    0]
 [1132   69   55    0    0   70    0]
 [   6    0    0    0    0    1    0]]


In [26]:
accuracy_score(true_classes, predicted_classes)

0.6542145593869731

# Calculating evaluation accurary

In [24]:
model_evaluate=model.evaluate_generator(generator=test_data)

In [25]:
print('Test loss:', model_evaluate[0])
print('Test accuracy:', model_evaluate[1])

Test loss: 0.016464771702885628
Test accuracy: 0.962180495262146


In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 16)        448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 28800)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 80)                2304080   
_________________________________________________________________
dropout_1 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                

# Extracting feautures from model

In [24]:
intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer('dropout_1').output)
intermediate_layer_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 62, 16)        448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 28800)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 80)                2304080   
_________________________________________________________________
dropout_1 (Dropout)          (None, 80)                0   

In [25]:
feauture_engg_data = intermediate_layer_model.predict(train_data)
feauture_engg_data = pd.DataFrame(feauture_engg_data)

In [26]:
feauture_test = intermediate_layer_model.predict(test_data)
feauture_test = pd.DataFrame(feauture_test)

In [17]:
feauture_test.to_csv('feauture_test_usage.csv')

In [18]:
feauture_engg_data.to_csv('feauture_engg_data_usage.csv')

In [15]:
feauture_engg_data = pd.read_csv('feauture_engg_data_usage.csv')
feauture_test = pd.read_csv('feauture_test_usage.csv')

In [16]:
feauture_engg_data = pd.concat([X, feauture_engg_data], axis = 1)
feauture_test = pd.concat([test, feauture_test], axis = 1)